# IMPORT DATA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

df = pd.read_csv('result/data/melting_point_features.csv')

y = df['Tm']
X = df.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=2601)

base_model = LGBMRegressor(random_state=2601, n_jobs=1)

# REMOVE OUTLIERS

In [ ]:
import numpy as np

def remove_outliers_advanced(df, col):
    initial_len = len(df)
    df = df[(df[col] > 0) & (df[col] < 4000)].copy()
    print(f"Đã xóa {initial_len - len(df)} mẫu phi vật lý (<=0 hoặc >4000).")

    log_data = np.log1p(df[col])
    
    Q1 = log_data.quantile(0.25)
    Q3 = log_data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound_log = Q1 - 2.0 * IQR
    upper_bound_log = Q3 + 2.0 * IQR
    
    lower_bound = np.expm1(lower_bound_log)
    upper_bound = np.expm1(upper_bound_log)
    
    print(f"Ngưỡng giữ lại: {lower_bound:.2f} đến {upper_bound:.2f}")

    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

train_data = X_train.copy()
train_data['Tm'] = y_train

print(f"Original Train Size: {len(train_data)}")

train_data_clean = remove_outliers_advanced(train_data, 'Tm')

print(f"Cleaned Train Size: {len(train_data_clean)}")
print(f"Removed Total: {len(train_data) - len(train_data_clean)} Samp")

X_train_clean = train_data_clean.drop(columns=['Tm'])
y_train_clean = train_data_clean['Tm']

X_train = X_train_clean
y_train = y_train_clean

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

print("\n✅ ĐÃ CẬP NHẬT THÀNH CÔNG!")
print(f"X_train shape mới: {X_train.shape}")
print(f"y_train shape mới: {y_train.shape}")

# RFECV Train

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
import joblib
import warnings

warnings.filterwarnings('ignore')

print("\n--- START RFECV ---")
start = time.time()

model_rfe = LGBMRegressor(
    objective='regression',
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=2601,
    n_jobs=-1,
    verbose=-1
)

rfe = RFECV(
    estimator=model_rfe, 
    min_features_to_select=50,
    step=20,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

rfe.fit(X_train, y_train)

selected_rfe = X_train.columns[rfe.support_]
print(f"Time Run: {time.time() - start:.2f} s")
print(f"RFECV Selected: {len(selected_rfe)} features")

joblib.dump(list(selected_rfe), 'result/rfe_features.pkl')

eval_model = LGBMRegressor(
    n_estimators=3000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.7,
    random_state=2601,
    n_jobs=-1, 
    verbose=-1
)

eval_model.fit(X_train[selected_rfe], y_train)

y_pred_log = eval_model.predict(X_test[selected_rfe])

y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n--- RESULT (REAL SCALE - KELVIN) ---")
print(f"Features: {len(selected_rfe)}")
print(f"MAE     : {mae:.4f} K")
print(f"RMSE    : {rmse:.4f} K")
print(f"R2      : {r2:.4f}")

# GENETIC ALGORITHM (GA) Train

In [ ]:
import warnings
import time
import joblib
import numpy as np
import pandas as pd
from sklearn_genetic import GAFeatureSelectionCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')

print("\n--- 🧬 START RUN GENETIC ALGORITHM ---")
start = time.time()

y_train_log = np.log1p(y_train)

X_train = X_train.replace([np.inf, -np.inf], np.nan)

model_ga = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    max_depth=-1,
    random_state=2601,
    n_jobs=1, 
    verbose=-1
)

ga = GAFeatureSelectionCV(
    estimator=model_ga,
    cv=3,
    scoring="neg_mean_absolute_error",
    population_size=50,    
    generations=15,
    mutation_probability=0.1,
    crossover_probability=0.8,
    keep_top_k=2,
    elitism=True,
    n_jobs=-1,
    verbose=True
)

ga.fit(X_train, y_train_log)

selected_ga = X_train.columns[ga.support_]

print(f"Time Run: {time.time() - start:.2f} s")
print(f"\n✅ GA Chosen {len(selected_ga)} features")
joblib.dump(list(selected_ga), 'result/ga_features.pkl')

print("\n--- EVALUATING (REAL SCALE) ---")
eval_model = LGBMRegressor(
    n_estimators=3000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.7,
    random_state=2601,
    n_jobs=-1, 
    verbose=-1
)

eval_model.fit(X_train[selected_ga], y_train_log)

y_pred_log = eval_model.predict(X_test[selected_ga])

y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n--- RESULT ---")
print(f"Features: {len(selected_ga)}")
print(f"MAE     : {mae:.4f} K")
print(f"RMSE    : {rmse:.4f} K")
print(f"R2      : {r2:.4f}")

# UNION 2 MODEL

In [ ]:
import joblib

rfe_features = joblib.load('result/rfe_features.pkl')
ga_features = joblib.load('result/ga_features.pkl')

common_features = set(rfe_features) | set(ga_features)

print(f"\n💎 Tổng features sau khi gộp (Union): {len(common_features)}")
print(list(common_features))

# CHOICE BEST FEATURES

In [ ]:
import joblib
from lightgbm import LGBMRegressor

best_features = list(common_features)

print(f"✅ Đang train model với {len(best_features)} features...")

manual_params = {
    'n_estimators': 5000,
    'learning_rate': 0.001,
    'num_leaves': 31,
    'max_depth': 15,
    'objective': 'regression_l1',
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'random_state': 2601,
    'n_jobs': 1,
    'verbose': -1
}

final_model = LGBMRegressor(**manual_params)

final_model.fit(X_clean[best_features], y)

# Lưu model và danh sách features
joblib.dump(final_model, 'result/final_melting_point_model.pkl')
joblib.dump(best_features, 'result/final_features_list.pkl')

print("💾 Đã lưu model và features thành công!")

# SCORING

## Score Combine Features

In [ ]:
import pandas as pd
import numpy as np
import joblib
import ast
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

model = joblib.load('result/final_melting_point_model.pkl')
features = joblib.load('result/final_features_list.pkl')

df = pd.read_csv('result/data/melting_point_features.csv')

needed_cols = list(features) + ['Tm']

existing_cols = [c for c in needed_cols if c in df.columns]

df_reduced = df[existing_cols].copy()

del df
gc.collect()

y = df_reduced['Tm']
X = df_reduced.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.mask(X > 1e308, np.nan)

print("⚙️(Imputing)...")
imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

_, X_test, _, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=2601)

y_pred = model.predict(X_test[features])

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- 🏁 RESULT ---")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

## Score each Model

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')

print("--- 📊 THỐNG KÊ DỮ LIỆU GỐC ---")
print(df['Tm'].describe())

neg_count = (df['Tm'] <= 0).sum()
print(f"\nSố lượng mẫu có Tm <= 0: {neg_count} mẫu")

df = df.dropna(subset=['Tm'])

df_clean = df[(df['Tm'] > 0) & (df['Tm'] < 4000)].copy()
print(f"✅ Dữ liệu sau khi lọc Outliers vô lý: {len(df_clean)} (Đã xóa {len(df) - len(df_clean)} dòng)")

y = df_clean['Tm']
X = df_clean.drop(columns=['Tm']).select_dtypes(include=[np.number])

X.replace([np.inf, -np.inf], np.nan, inplace=True)
thresh = len(X) * 0.3 
X = X.dropna(thresh=thresh, axis=1)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

y_log = np.log1p(y)

X_train, X_test, y_train_log, y_test_log = train_test_split(X_clean, y_log, test_size=0.2, random_state=2601)

def get_metrics(name, feature_list):
    valid_feats = [f for f in feature_list if f in X_train.columns]

    if not valid_feats: return {"Method": name, "Features": 0, "RMSE": 0, "R2": 0, "MAE": 0}

    model = LGBMRegressor(
        n_jobs=-1,
        verbose=-1,
        random_state=2601,
        n_estimators=5000,      
        learning_rate=0.01,      
        num_leaves=31,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.6,
        reg_alpha=0.5,
        reg_lambda=0.5
    )
    
    callbacks = [lgb.early_stopping(stopping_rounds=200, verbose=False), lgb.log_evaluation(period=0)]

    model.fit(X_train[valid_feats], y_train_log, 
              eval_set=[(X_test[valid_feats], y_test_log)],
              eval_metric='rmse',
              callbacks=callbacks)

    y_pred_log = model.predict(X_test[valid_feats], num_iteration=model.best_iteration_)
    
    y_pred = np.expm1(y_pred_log)
    y_test_real = np.expm1(y_test_log)

    return {
        "Method": name,
        "Features": len(valid_feats),
        "RMSE": np.sqrt(mean_squared_error(y_test_real, y_pred)),
        "R2": r2_score(y_test_real, y_pred),
        "MAE": mean_absolute_error(y_test_real, y_pred)
    }

feats_all = list(X_train.columns)

try:
    feats_rfe = joblib.load('result/rfe_features.pkl')
except:
    feats_rfe = []

try:
    feats_ga = joblib.load('result/ga_features.pkl')
except:
    feats_ga = []

results = []
results.append(get_metrics("Original", feats_all))
results.append(get_metrics("RFE", feats_rfe))
results.append(get_metrics("GA", feats_ga))

df_res = pd.DataFrame(results)
base_rmse = df_res.loc[0, 'RMSE']
base_r2 = df_res.loc[0, 'R2']
base_mae = df_res.loc[0, 'MAE']

df_res['Diff_RMSE'] = df_res['RMSE'] - base_rmse
df_res['Diff_R2'] = df_res['R2'] - base_r2
df_res['Diff_MAE'] = df_res['MAE'] - base_mae

print("\n--- KẾT QUẢ MỚI ---")
print(df_res.round(4))

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(y, kde=True, color='blue').set_title("Phân phối Tm gốc")
plt.subplot(1, 2, 2)
sns.histplot(y_log, kde=True, color='green').set_title("Phân phối Tm sau Log (Chuẩn hơn)")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')
df = df.dropna(subset=['Tm'])

df = df[(df['Tm'] > 0) & (df['Tm'] < 4000)].copy()

y_log_temp = np.log1p(df['Tm'])
Q1 = y_log_temp.quantile(0.25)
Q3 = y_log_temp.quantile(0.75)
IQR = Q3 - Q1
lower_bound_log = Q1 - 2.0 * IQR
upper_bound_log = Q3 + 2.0 * IQR

lower_bound = np.expm1(lower_bound_log)
upper_bound = np.expm1(upper_bound_log)

df_clean = df[(df['Tm'] >= lower_bound) & (df['Tm'] <= upper_bound)].copy()

print(f"✅ Dữ liệu sạch cuối cùng: {len(df_clean)} mẫu (Đã loại bỏ {len(df) - len(df_clean)} nhiễu)")

y = df_clean['Tm']
X = df_clean.drop(columns=['Tm']).select_dtypes(include=[np.number])

y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

X.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='median')
X_final = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_log, test_size=0.2, random_state=2601)

def get_metrics(name, feature_list):
    valid_feats = [f for f in feature_list if f in X_train.columns]

    if not valid_feats: 
        return {"Method": name, "Features": 0, "R2": 0, "MAE": 0, "RMSE": 0}

    model = LGBMRegressor(
        n_jobs=-1,
        verbose=-1,
        random_state=2601,
        n_estimators=5000,      
        learning_rate=0.01,     
        num_leaves=31,          
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.7,   
        reg_alpha=0.1,
        reg_lambda=0.1
    )
    
    callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=False), lgb.log_evaluation(period=0)]

    model.fit(X_train[valid_feats], y_train, 
              eval_set=[(X_test[valid_feats], y_test)],
              eval_metric='rmse',
              callbacks=callbacks)

    y_pred_log = model.predict(X_test[valid_feats], num_iteration=model.best_iteration_)
    y_pred_real = np.expm1(y_pred_log)
    y_test_real = np.expm1(y_test)

    return {
        "Method": name,
        "Features": len(valid_feats),
        "R2": r2_score(y_test_real, y_pred_real),
        "MAE": mean_absolute_error(y_test_real, y_pred_real),
        "RMSE": np.sqrt(mean_squared_error(y_test_real, y_pred_real))
    }

feats_all = list(X_train.columns)

try:
    feats_rfe = joblib.load('result/rfe_features.pkl')
except:
    print("⚠️ Không tìm thấy file RFE, bỏ qua.")
    feats_rfe = []

try:
    feats_ga = joblib.load('result/ga_features.pkl')
except:
    print("⚠️ Không tìm thấy file GA, bỏ qua.")
    feats_ga = []

results = []
print("🚀 Đang chạy đánh giá Original...")
results.append(get_metrics("Original", feats_all))

if feats_rfe:
    print("🚀 Đang chạy đánh giá RFE...")
    results.append(get_metrics("RFE", feats_rfe))

if feats_ga:
    print("🚀 Đang chạy đánh giá GA...")
    results.append(get_metrics("GA", feats_ga))

df_res = pd.DataFrame(results)

if not df_res.empty:
    base_r2 = df_res.loc[0, 'R2']
    base_mae = df_res.loc[0, 'MAE']
    base_rmse = df_res.loc[0, 'RMSE']

    df_res['Diff_R2'] = df_res['R2'] - base_r2
    df_res['Diff_MAE'] = df_res['MAE'] - base_mae
    df_res['Diff_RMSE'] = df_res['RMSE'] - base_rmse

    print("\n" + "="*40)
    print("📊 BẢNG KẾT QUẢ SO SÁNH (Trên tập Real Tm)")
    print("="*40)
    print(df_res.round(4))
    
    common = set(feats_rfe) & set(feats_ga)
    print(f"\n💎 Common Features ({len(common)}):", list(common))
else:
    print("Không có kết quả nào được ghi nhận.")

# GridSearch Find Best Params

In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from optuna.integration import LightGBMPruningCallback

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'mae',
        'verbosity': -1,
        'n_jobs': 4,
        'random_state': 2601,
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 40),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 50),
    }
    
    if 'best_features' in globals():
        features_to_use = list(best_features)
    elif 'selected_ga' in globals():
        features_to_use = list(selected_ga)
    else:
        features_to_use = list(X_train_clean.columns)

    X_opt = X_train_clean[features_to_use]
    y_opt = y_train_clean
    
    cv = KFold(n_splits=3, shuffle=True, random_state=2601)
    scores = []
    
    for train_idx, val_idx in cv.split(X_opt, y_opt):
        X_tr, X_val = X_opt.iloc[train_idx], X_opt.iloc[val_idx]
        y_tr, y_val = y_opt.iloc[train_idx], y_opt.iloc[val_idx]
        
        model = lgb.LGBMRegressor(**param)
        
        pruning_callback = LightGBMPruningCallback(trial, "l1")
        
        model.fit(
            X_tr, y_tr, 
            eval_set=[(X_val, y_val)], 
            callbacks=[
                lgb.early_stopping(stopping_rounds=20, verbose=False),
                pruning_callback
            ]
        )
        
        preds = model.predict(X_val)
        scores.append(mean_absolute_error(y_val, preds))
    
    return np.mean(scores)

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=5, timeout=60)

print('Best params:', study.best_params)

# Elbow GA Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from lightgbm import LGBMRegressor

best_params = {
    'learning_rate': 0.01,
    'n_estimators': 2000,
    'num_leaves': 50,
    'max_depth': -1,
    'random_state': 2601,
    'n_jobs': 1,
    'verbose': -1
}

valid_ga_feats = [f for f in list(selected_ga) if f in X_train.columns]

print("Đang xếp hạng features...")
ranker = LGBMRegressor(**best_params)
ranker.fit(X_train[valid_ga_feats], y_train)

imp_df = pd.DataFrame({
    'Feature': valid_ga_feats,
    'Importance': ranker.feature_importances_
}).sort_values(by='Importance', ascending=False)

sorted_feats = imp_df['Feature'].tolist()

steps = list(range(len(sorted_feats), 99, -50)) + list(range(90, 0, -10))
results = []

print(f"\nBắt đầu vòng lặp cắt giảm features ({len(steps)} vòng)...")

for k in steps:
    current_feats = sorted_feats[:k]
    
    model = LGBMRegressor(**best_params)
    model.fit(X_train[current_feats], y_train)
    
    y_pred_log = model.predict(X_test[current_feats])
    
    y_pred_real = np.expm1(y_pred_log)
    y_test_real = np.expm1(y_test)

    r2 = r2_score(y_test_real, y_pred_real)
    rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
    mae = mean_absolute_error(y_test_real, y_pred_real)

    print(f"   -> Dùng {k:3d} features: R2 = {r2:.4f} | MAE = {mae:.2f} | RMSE = {rmse:.2f}")
    results.append({'Num_Features': k, 'R2': r2, 'MAE': mae, 'RMSE': rmse})

df_results = pd.DataFrame(results).sort_values(by='Num_Features')
df_leaderboard = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)
csv_filename = 'result/feature_selection_results.csv'
df_results.to_csv(csv_filename, index=False)

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['R2'], marker='o', linewidth=2, color='blue')

best_row = df_results.loc[df_results['R2'].idxmax()]
plt.scatter(best_row['Num_Features'], best_row['R2'], color='red', s=150, zorder=5)
plt.annotate(f"Đỉnh: {best_row['R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['R2']), 
             xytext=(best_row['Num_Features']+20, best_row['R2']-0.01),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('Biểu đồ Elbow: Hiệu quả khi giảm dần số lượng Features', fontsize=14)
plt.xlabel('Số lượng Features', fontsize=12)
plt.ylabel('Độ chính xác (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_xaxis()
plt.show()

print("\nBẢNG XẾP HẠNG (SCORE GIẢM DẦN):")
print(df_leaderboard[['R2', 'Num_Features', 'MAE', 'RMSE']].head(10))

print("\nBẢNG THEO THỨ TỰ FEATURE (ÍT -> NHIỀU):")
print(df_results[['Num_Features', 'R2', 'MAE', 'RMSE']].head(10))

# Elbow RFECV Model

In [ ]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

print("\n--- START RFE WITH LEADERBOARD ---")
start = time.time()

model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=1,
    verbose=-1
)

rfecv = RFECV(
    estimator=model,
    step=20,
    cv=3,
    scoring='r2', 
    min_features_to_select=50,
    n_jobs=-1,
    verbose=1
)

rfecv.fit(X_train, y_train)

r2_scores = rfecv.cv_results_['mean_test_score']
n_scores = len(r2_scores)

feature_counts = [50 + i * 20 for i in range(n_scores)]

if len(feature_counts) > len(r2_scores):
    feature_counts = feature_counts[:len(r2_scores)]

df_results = pd.DataFrame({
    'Num_Features': feature_counts,
    'Score_R2': r2_scores
})

df_leaderboard = df_results.sort_values(by='Score_R2', ascending=False).reset_index(drop=True)

selected_rfecv = X_train.columns[rfecv.support_]
print(f"\nTime Run: {time.time() - start:.2f} s")
print(f"Best Number of Features: {rfecv.n_features_}")
print(f"Best CV R2 Score: {df_leaderboard.iloc[0]['Score_R2']:.4f}")

print("\nLEADERBOARD (DESCENDING SCORE):")
print(df_leaderboard.head(10).to_string(index=False))

print("\nPROGRESS (BY FEATURE COUNT):")
print(df_results.head(10).to_string(index=False))

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['Score_R2'], marker='o', color='green', linewidth=2)

best_row = df_leaderboard.iloc[0]
plt.scatter(best_row['Num_Features'], best_row['Score_R2'], color='red', s=150, zorder=5)
plt.annotate(f"Best: {best_row['Score_R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['Score_R2']), 
             xytext=(best_row['Num_Features']+20, best_row['Score_R2']-0.005),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('RFECV Performance Curve', fontsize=14)
plt.xlabel('Number of Features Selected', fontsize=12)
plt.ylabel('Cross Validation Score (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nSelected Features List:")
print(list(selected_rfecv))

y_pred_log = rfecv.predict(X_test)
y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

test_r2 = r2_score(y_test_real, y_pred_real)
test_mae = mean_absolute_error(y_test_real, y_pred_real)
test_rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("\n--- FINAL TEST EVALUATION (Best Features) ---")
print(f"R2 Score : {test_r2:.4f}")
print(f"MAE      : {test_mae:.4f} K")
print(f"RMSE     : {test_rmse:.4f} K")

df_results.to_csv('result/rfecv_results.csv', index=False)

# Compare before and after using partial correlation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ULTRA_HIGH_CORR = 0.998

print("⏳ Đang tính toán ma trận tương quan...")
corr_matrix = X_train.corr().abs()

ranker_temp = LGBMRegressor(n_estimators=100, verbose=-1, random_state=2601)
ranker_temp.fit(X_train, y_train)
importances = pd.Series(ranker_temp.feature_importances_, index=X_train.columns)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = []

for column in upper.columns:
    correlated_cols = upper.index[upper[column] > ULTRA_HIGH_CORR].tolist()
    
    if correlated_cols:
        for other_col in correlated_cols:
            if other_col in to_drop: continue
            
            imp_col = importances.get(column, 0)
            imp_other = importances.get(other_col, 0)
            
            if imp_col < imp_other:
                to_drop.append(column)
                break 
            else:
                to_drop.append(other_col)

to_drop = list(set(to_drop))
print(f"✂️ Đã tìm thấy {len(to_drop)} features trùng lặp (Corr > {ULTRA_HIGH_CORR})")

if len(to_drop) > 0:
    feats_filtered = [f for f in X_train.columns if f not in to_drop]
    
    print(f"🚀 Đang chạy đánh giá lại với {len(feats_filtered)} features...")
    res_filtered = get_metrics(f"Filtered (Corr > {ULTRA_HIGH_CORR})", feats_filtered)
    
    new_r2 = res_filtered['R2']
    
    old_r2 = df_res.loc[df_res['Method'] == 'Original', 'R2'].values[0]
    
    print(f"\n✅ Kết quả R2 Cũ (Original): {old_r2:.4f}")
    print(f"✅ Kết quả R2 Mới (Filtered): {new_r2:.4f}")
    
    methods = ['Original', 'Filtered']
    scores = [old_r2, new_r2]
    colors = ['gray', 'green' if new_r2 >= old_r2 else 'red']

    plt.figure(figsize=(8, 6))
    bars = plt.bar(methods, scores, color=colors, width=0.5)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.4f}',
                 ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.title(f'So sánh R2: Giữ nguyên vs Lọc Tương quan (> {ULTRA_HIGH_CORR})', fontsize=14)
    plt.ylabel('R2 Score (Real Scale)', fontsize=12)
    plt.ylim(0, max(scores) + 0.1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    new_row = pd.DataFrame([res_filtered])
    df_res = pd.concat([df_res, new_row], ignore_index=True)

else:
    print("✅ Không có features nào quá giống nhau để xóa.")

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')
df = df.dropna(subset=['Tm'])

df = df[(df['Tm'] > 0) & (df['Tm'] < 1000)].copy()

print(f"Data size after removing outliers (>1000K): {len(df)}")

y = df['Tm']
X = df.drop(columns=['Tm']).select_dtypes(include=[np.number])

y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

X.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='median')
X_final = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_log, test_size=0.2, random_state=2601)

corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.999)]

if to_drop:
    X_train = X_train.drop(columns=to_drop)
    X_test = X_test.drop(columns=to_drop)
    print(f"Dropped {len(to_drop)} high correlation features")

voting_model = LGBMRegressor(
    learning_rate=0.01,
    n_estimators=2000,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=-1,
    verbose=-1
)

voting_model.fit(X_train, y_train)

y_pred_log = voting_model.predict(X_test)
y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

new_r2 = r2_score(y_test_real, y_pred_real)
mae_real = mean_absolute_error(y_test_real, y_pred_real)
rmse_real = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("\n--- RESULT ---")
print(f"R2 Score : {new_r2:.4f}")
print(f"MAE      : {mae_real:.4f} K")
print(f"RMSE     : {rmse_real:.4f} K")

plt.figure(figsize=(10, 8))
plt.scatter(y_test_real, y_pred_real, alpha=0.4, color='blue', s=15, label='Data')

max_val = max(y_test_real.max(), y_pred_real.max())
min_val = min(y_test_real.min(), y_pred_real.min())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Fit')

plt.title(f'Evaluation (Tm < 1000K)\nR2 = {new_r2:.4f} | MAE = {mae_real:.2f} K', fontsize=14)
plt.xlabel('Actual Tm (K)', fontsize=12)
plt.ylabel('Predicted Tm (K)', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

joblib.dump(voting_model, 'result/model/final_melting_point_model.pkl')
joblib.dump(list(X_train.columns), 'result/model/final_features.pkl')

### Phân tích Kết quả Thực nghiệm (Phạm vi )

**1. Kết quả Định lượng:**
Sau khi loại bỏ các giá trị ngoại lai () và lọc bỏ các đặc trưng tương quan cao, mô hình đạt:

* ****: Mức độ giải thích biến thiên dữ liệu ở mức trung bình khá.
* ****: Sai số tuyệt đối trung bình khoảng 65 độ.

**2. Đánh giá Nguyên nhân Biến động:**
So với thử nghiệm trên tập dữ liệu toàn phần (bao gồm cả chất vô cơ/muối), các chỉ số này phản ánh chính xác hiệu năng trên nhóm hợp chất hữu cơ:

* **Hiện tượng Hạn chế Phạm vi (Range Restriction):** Việc loại bỏ miền giá trị cao () làm giảm phương sai tổng thể của tập dữ liệu. Về mặt toán học, điều này khiến  giảm tự nhiên, đòi hỏi mô hình phải có độ nhạy cao hơn để phân biệt các mẫu có nhiệt độ gần nhau.
* **Độ chính xác thực tế (Honest Baseline):** Mức  là sai số thực tế khi loại bỏ ảnh hưởng của các điểm dữ liệu cực đoan (outliers). Biểu đồ phân tán cho thấy hiện tượng "co về trung bình" (regression to the mean), khi mô hình có xu hướng dự đoán thấp hơn thực tế ở vùng nhiệt độ cao (600-800K) và cao hơn thực tế ở vùng nhiệt độ thấp (<200K).

**Kết luận:** Đây là kết quả cơ sở (baseline) tin cậy cho bài toán dự đoán trên hợp chất hữu cơ, loại bỏ hoàn toàn các yếu tố gây nhiễu hoặc "ảo giác thống kê" từ các giá trị ngoại lai.

---